In [8]:
# order_book_sim_extended.py
# ---------------------------------------------------------
# Limit order book simulator with extended order types.
# Supported:
#   - Order types: limit, market, stop (market), stop-limit
#   - TIF: GTC (default), IOC, FOK
#   - Post-Only for limit orders (reject if it would cross)
# Matching:
#   - Price-time priority, partial fills
#   - Trades occur at the resting order's price
# Stops:
#   - Trigger on last trade price; if none yet, use best quotes
# Random flow:
#   - Generates a mix of these orders over time
# ---------------------------------------------------------

from __future__ import annotations
from dataclasses import dataclass
from collections import deque
import heapq
import itertools
import random
from typing import Deque, Dict, List, Optional, Tuple

# ---------- Configuration ----------
TICK_SIZE = 0.5
INIT_LEVELS = 6
INIT_QTY_RANGE = (20, 120)

ARRIVAL_STEPS = 140  # number of events

# Random order flow knobs
P_LIMIT = 0.70          # else market (for non-stop arrivals)
P_STOP = 0.10           # probability to create a stop-market
P_STOP_LIMIT = 0.10     # probability to create a stop-limit
P_POST_ONLY = 0.10      # for limit orders
TIF_WEIGHTS = {         # draw among these for all active orders
    "GTC": 0.70,
    "IOC": 0.20,
    "FOK": 0.10,
}

QTY_RANGE = (5, 80)
CROSSING_BIAS = 0.30    # chance a limit is placed to cross
SEED = 7
SHOW_LEVELS = 6

# ---------- Utilities ----------
_order_id = itertools.count(1)

def round_to_tick(price: float) -> float:
    """Snap a float price to the nearest tick."""
    ticks = int(round(price / TICK_SIZE))
    return ticks * TICK_SIZE


@dataclass
class Order:
    """Generic order record used for both active and resting orders."""
    id: int
    side: str              # 'buy' or 'sell'
    qty: int
    type: str              # 'limit', 'market', 'stop', 'stop_limit'
    price: Optional[float] # limit price (if applicable)
    ts: int                # event time (discrete integer)
    tif: str = "GTC"       # 'GTC', 'IOC', 'FOK'
    post_only: bool = False
    stop_price: Optional[float] = None  # for stop / stop-limit


class OrderBook:
    """
    Price-time priority LOB with:
    - FIFO queues per price
    - Min-heap for asks, max-heap for bids (via negative prices)
    - Stop orders stored off-book until triggered
    """

    def __init__(self):
        self.bids: Dict[float, Deque[Order]] = {}
        self.asks: Dict[float, Deque[Order]] = {}
        self._bid_heap: List[float] = []
        self._ask_heap: List[float] = []
        self.stop_orders: List[Order] = []  # waiting for trigger

        # trade record: (ts, price, qty, buy_id, sell_id)
        self.trades: List[Tuple[int, float, int, int, int]] = []
        self.last_trade_price: Optional[float] = None

    # ----- Best price helpers (with lazy cleanup) -----
    def _best_bid(self) -> Optional[float]:
        while self._bid_heap:
            p = -self._bid_heap[0]
            q = self.bids.get(p)
            if q and len(q) > 0:
                return p
            heapq.heappop(self._bid_heap)
        return None

    def _best_ask(self) -> Optional[float]:
        while self._ask_heap:
            p = self._ask_heap[0]
            q = self.asks.get(p)
            if q and len(q) > 0:
                return p
            heapq.heappop(self._ask_heap)
        return None

    # ----- Resting insertion -----
    def _rest(self, o: Order) -> None:
        """Add a limit order to the book (rest any remainder)."""
        assert o.type == "limit", "Only limit orders can rest on book."
        book = self.bids if o.side == "buy" else self.asks
        heap = self._bid_heap if o.side == "buy" else self._ask_heap
        price = round_to_tick(float(o.price))
        o.price = price
        if price not in book:
            book[price] = deque()
            heapq.heappush(heap, -price if o.side == "buy" else price)
        book[price].append(o)

    # ----- Liquidity check for FOK -----
    def _cumulative_liquidity(self, taker_side: str, limit_price: Optional[float]) -> int:
        """
        Return total opposing liquidity available at prices that would execute
        this taker order (side = taker side). For MARKET, limit_price is None.
        """
        if taker_side == "buy":
            prices = sorted(self.asks.keys())
            total = 0
            for p in prices:
                if limit_price is not None and p > limit_price:
                    break
                total += sum(q.qty for q in self.asks[p])
            return total
        else:
            prices = sorted(self.bids.keys(), reverse=True)
            total = 0
            for p in prices:
                if limit_price is not None and p < limit_price:
                    break
                total += sum(q.qty for q in self.bids[p])
            return total

    # ----- Stop management -----
    def queue_stop(self, o: Order) -> None:
        """Hold stop/stop-limit until trigger condition is met."""
        self.stop_orders.append(o)

    def collect_triggered_stops(self, ts: int) -> List[Order]:
        """Return and remove stops that should trigger now."""
        triggered: List[Order] = []
        if not self.stop_orders:
            return triggered

        bb = self._best_bid()
        ba = self._best_ask()

        # Use last trade price if available, else best quotes (touch trigger)
        ref_buy = self.last_trade_price if self.last_trade_price is not None else ba
        ref_sell = self.last_trade_price if self.last_trade_price is not None else bb

        keep: List[Order] = []
        for o in self.stop_orders:
            ok = False
            if o.side == "buy" and ref_buy is not None and o.stop_price is not None:
                ok = ref_buy >= o.stop_price
            elif o.side == "sell" and ref_sell is not None and o.stop_price is not None:
                ok = ref_sell <= o.stop_price

            if ok:
                # Convert to active order
                if o.type == "stop":
                    new_o = Order(o.id, o.side, o.qty, "market", None, ts, o.tif, False, None)
                else:
                    # stop-limit: price is the limit price already carried by 'o.price'
                    new_o = Order(o.id, o.side, o.qty, "limit", o.price, ts, o.tif, o.post_only, None)
                triggered.append(new_o)
            else:
                keep.append(o)

        self.stop_orders = keep
        return triggered

    # ----- Core matching engine -----
    def add(self, order: Order) -> List[Tuple[int, float, int, int, int]]:
        """
        Insert an order (active or stop). Returns list of trades:
        (ts, trade_price, trade_qty, buy_id, sell_id)
        """
        trades: List[Tuple[int, float, int, int, int]] = []

        def record(px: float, q: int, buy_id: int, sell_id: int):
            self.trades.append((order.ts, px, q, buy_id, sell_id))
            trades.append((order.ts, px, q, buy_id, sell_id))
            self.last_trade_price = px

        # Handle stop orders: keep off-book until triggered
        if order.type in ("stop", "stop_limit"):
            self.queue_stop(order)
            return trades  # no immediate trade

        # MARKET orders: always IOC-like by nature
        if order.type == "market":
            qty_left = order.qty
            if order.tif == "FOK":
                need = order.qty
                avail = self._cumulative_liquidity(order.side, None)
                if avail < need:
                    return trades  # reject (no partial fills for FOK)

            if order.side == "buy":
                while qty_left > 0:
                    best_ask = self._best_ask()
                    if best_ask is None:
                        break
                    q = self.asks[best_ask][0]
                    traded = min(qty_left, q.qty)
                    q.qty -= traded
                    qty_left -= traded
                    record(best_ask, traded, buy_id=order.id, sell_id=q.id)
                    if q.qty == 0:
                        self.asks[best_ask].popleft()
            else:
                while qty_left > 0:
                    best_bid = self._best_bid()
                    if best_bid is None:
                        break
                    q = self.bids[best_bid][0]
                    traded = min(qty_left, q.qty)
                    q.qty -= traded
                    qty_left -= traded
                    record(best_bid, traded, buy_id=q.id, sell_id=order.id)
                    if q.qty == 0:
                        self.bids[best_bid].popleft()
            return trades  # leftover discarded

        # LIMIT orders
        assert order.price is not None
        px = round_to_tick(float(order.price))
        order.price = px

        # Post-Only: if the order would cross, reject it entirely
        if order.post_only:
            bb = self._best_bid()
            ba = self._best_ask()
            would_cross = (
                (order.side == "buy" and ba is not None and px >= ba) or
                (order.side == "sell" and bb is not None and px <= bb)
            )
            if would_cross:
                return trades  # silently canceled (post-only)

        # FOK pre-check: is there enough opposing liquidity at/better than px?
        if order.tif == "FOK":
            limit_for_taker = px
            avail = self._cumulative_liquidity(order.side, limit_for_taker)
            if avail < order.qty:
                return trades  # reject entirely

        qty_left = order.qty

        if order.side == "buy":
            # Try to cross as long as best ask <= limit price
            while qty_left > 0:
                best_ask = self._best_ask()
                if best_ask is None or px < best_ask:
                    break
                q = self.asks[best_ask][0]
                traded = min(qty_left, q.qty)
                q.qty -= traded
                qty_left -= traded
                record(best_ask, traded, buy_id=order.id, sell_id=q.id)
                if q.qty == 0:
                    self.asks[best_ask].popleft()

            # Rest remainder if allowed by TIF
            if qty_left > 0 and order.tif != "IOC":
                self._rest(Order(order.id, order.side, qty_left, "limit", px, order.ts, order.tif, order.post_only))

        else:
            while qty_left > 0:
                best_bid = self._best_bid()
                if best_bid is None or px > best_bid:
                    break
                q = self.bids[best_bid][0]
                traded = min(qty_left, q.qty)
                q.qty -= traded
                qty_left -= traded
                record(best_bid, traded, buy_id=q.id, sell_id=order.id)
                if q.qty == 0:
                    self.bids[best_bid].popleft()

            if qty_left > 0 and order.tif != "IOC":
                self._rest(Order(order.id, order.side, qty_left, "limit", px, order.ts, order.tif, order.post_only))

        return trades

    # ----- Rendering & stats -----
    def top_of_book(self) -> Tuple[Optional[float], int, Optional[float], int]:
        bb = self._best_bid()
        ba = self._best_ask()
        bid_sz = sum(o.qty for o in self.bids.get(bb, [])) if bb is not None else 0
        ask_sz = sum(o.qty for o in self.asks.get(ba, [])) if ba is not None else 0
        return bb, bid_sz, ba, ask_sz

    def depth(self, side: str, n: int) -> List[Tuple[float, int]]:
        if side == "buy":
            prices = sorted(self.bids.keys(), reverse=True)
            book = self.bids
        else:
            prices = sorted(self.asks.keys())
            book = self.asks
        out = []
        for p in prices[:n]:
            out.append((p, sum(o.qty for o in book[p])))
        return out

    def mid(self) -> Optional[float]:
        bb, _, ba, _ = self.top_of_book()
        if bb is None or ba is None:
            return None
        return round_to_tick((bb + ba) / 2.0)

    def render(self, levels: int = 5) -> str:
        left = self.depth("buy", levels)
        right = self.depth("sell", levels)
        max_rows = max(len(left), len(right))
        left += [("", "")] * (max_rows - len(left))
        right += [("", "")] * (max_rows - len(right))
        lines = ["  BID (px x qty)        |        ASK (px x qty)"]
        for i in range(max_rows):
            lp = f"{left[i][0]:>7}" if left[i][0] != "" else "       "
            lq = f"{left[i][1]:>5}" if left[i][1] != "" else "     "
            rp = f"{right[i][0]:>7}" if right[i][0] != "" else "       "
            rq = f"{right[i][1]:>5}" if right[i][1] != "" else "     "
            lines.append(f" {lp} x {lq}        |        {rp} x {rq}")
        lines.append(f"\n  (Queued stops: {len(self.stop_orders)})")
        return "\n".join(lines)


# ---------- Initialization & random flow ----------
def initialize_random_book(book: OrderBook, mid: float, ts: int = 0) -> None:
    """Fill symmetric random depth around an initial mid."""
    for i in range(1, INIT_LEVELS + 1):
        bid_px = round_to_tick(mid - i * TICK_SIZE)
        ask_px = round_to_tick(mid + i * TICK_SIZE)
        bid_qty = random.randint(*INIT_QTY_RANGE)
        ask_qty = random.randint(*INIT_QTY_RANGE)
        bid = Order(next(_order_id), "buy", bid_qty, "limit", bid_px, ts)
        ask = Order(next(_order_id), "sell", ask_qty, "limit", ask_px, ts)
        book.add(bid)
        book.add(ask)


def draw_tif() -> str:
    r = random.random()
    acc = 0.0
    for k, w in TIF_WEIGHTS.items():
        acc += w
        if r <= acc:
            return k
    return "GTC"


def sample_arrival(book: OrderBook, ts: int) -> Order:
    """
    Generate a random incoming order making use of the extended features.
    - Sometimes emits stop or stop-limit orders.
    - Otherwise emits limit/market with IOC/FOK/GTC and optional Post-Only.
    """
    side = "buy" if random.random() < 0.5 else "sell"
    qty = random.randint(*QTY_RANGE)
    tif = draw_tif()

    bb, _, ba, _ = book.top_of_book()
    if bb is None and ba is None:
        anchor = 100.0
        bb = round_to_tick(anchor - TICK_SIZE)
        ba = round_to_tick(anchor + TICK_SIZE)

    # Decide stop vs regular
    r = random.random()
    if r < P_STOP:
        # STOP MARKET
        if side == "buy":
            # trigger above current ask
            stop_px = round_to_tick((ba if ba is not None else 100.0) + random.randint(1, 3)*TICK_SIZE)
        else:
            stop_px = round_to_tick((bb if bb is not None else 100.0) - random.randint(1, 3)*TICK_SIZE)
        return Order(next(_order_id), side, qty, "stop", None, ts, tif, False, stop_px)

    elif r < P_STOP + P_STOP_LIMIT:
        # STOP-LIMIT: trigger at stop, place limit at (often) the stop level
        if side == "buy":
            stop_px = round_to_tick((ba if ba is not None else 100.0) + random.randint(1, 3)*TICK_SIZE)
            limit_px = stop_px  # could also be stop_px + n*ticks
        else:
            stop_px = round_to_tick((bb if bb is not None else 100.0) - random.randint(1, 3)*TICK_SIZE)
            limit_px = stop_px
        return Order(next(_order_id), side, qty, "stop_limit", limit_px, ts, tif, False, stop_px)

    # Regular (non-stop): choose market vs limit
    typ = "limit" if random.random() < P_LIMIT else "market"

    if typ == "market":
        return Order(next(_order_id), side, qty, "market", None, ts, tif, False, None)

    # Limit: propose a price around mid, occasionally crossing
    mid = book.mid()
    if mid is None:
        mid = (bb + ba) / 2 if bb is not None and ba is not None else 100.0

    cross = random.random() < CROSSING_BIAS
    delta = random.randint(0, 4)
    if side == "buy":
        px = round_to_tick(mid - delta * TICK_SIZE)
        if cross and ba is not None:
            px = max(px, ba)  # touch/cross
    else:
        px = round_to_tick(mid + delta * TICK_SIZE)
        if cross and bb is not None:
            px = min(px, bb)

    post_only = (random.random() < P_POST_ONLY)
    return Order(next(_order_id), side, qty, "limit", px, ts, tif, post_only, None)


# ---------- Driver ----------
def simulate(steps: int = ARRIVAL_STEPS, seed: int = SEED) -> None:
    random.seed(seed)
    ob = OrderBook()
    start_mid = 100.0
    initialize_random_book(ob, start_mid, ts=0)

    print("\n--- INITIAL BOOK ---")
    print(ob.render(SHOW_LEVELS))
    bb, bs, ba, asz = ob.top_of_book()
    print(f"\nTop of book: bid {bb} x {bs} | ask {ba} x {asz}\n")

    t = 0
    for k in range(steps):
        t += 1
        incoming = sample_arrival(ob, ts=t)
        hdr = f"[t={t:03d}] IN: {incoming.type.upper():>10} {incoming.side.upper():>4} qty={incoming.qty}"
        if incoming.type in ("limit", "stop_limit"):
            hdr += f" @ {incoming.price}"
        if incoming.stop_price is not None:
            hdr += f" (stop={incoming.stop_price})"
        if incoming.tif != "GTC":
            hdr += f" [{incoming.tif}]"
        if incoming.post_only:
            hdr += " [POST-ONLY]"
        print(hdr)

        trades = ob.add(incoming)
        if trades:
            for (ts_, px, q, buy_id, sell_id) in trades:
                print(f"     -> TRADE ts={ts_} px={px} qty={q} (buy#{buy_id} vs sell#{sell_id})")
        else:
            if incoming.type in ("stop", "stop_limit"):
                print("     -> queued as STOP order")
            else:
                print("     -> no trade")

        # After processing, check for any stops that should trigger now.
        # Keep triggering in case cascading trades change prices further.
        while True:
            triggered = ob.collect_triggered_stops(ts=t)  # use same event time for simplicity
            if not triggered:
                break
            for trig in triggered:
                print(f"     TRIGGERED: {trig.type.upper()} {trig.side.upper()} qty={trig.qty}"
                      + (f" @ {trig.price}" if trig.type == "limit" else " (MKT)")
                      + (f" [{trig.tif}]" if trig.tif != "GTC" else ""))
                t_trades = ob.add(trig)
                if t_trades:
                    for (ts_, px, q, buy_id, sell_id) in t_trades:
                        print(f"         -> TRADE ts={ts_} px={px} qty={q} (buy#{buy_id} vs sell#{sell_id})")

        bb, bs, ba, asz = ob.top_of_book()
        mid = ob.mid()
        print(f"     Top: bid {bb} x {bs} | ask {ba} x {asz} | mid={mid}")

        if k % 10 == 0 or k < 3:
            print(ob.render(SHOW_LEVELS))

    print("\n--- FINAL BOOK ---")
    print(ob.render(SHOW_LEVELS))
    print(f"\nTotal trades: {len(ob.trades)}")


if __name__ == "__main__":
    simulate()



--- INITIAL BOOK ---
  BID (px x qty)        |        ASK (px x qty)
    99.5 x    61        |          100.5 x    39
    99.0 x    70        |          101.0 x   103
    98.5 x    26        |          101.5 x    29
    98.0 x    88        |          102.0 x    32
    97.5 x    66        |          102.5 x    94
    97.0 x    27        |          103.0 x    84

  (Queued stops: 0)

Top of book: bid 99.5 x 61 | ask 100.5 x 39

[t=001] IN:       STOP  BUY qty=16 (stop=101.0)
     -> queued as STOP order
     Top: bid 99.5 x 61 | ask 100.5 x 39 | mid=100.0
  BID (px x qty)        |        ASK (px x qty)
    99.5 x    61        |          100.5 x    39
    99.0 x    70        |          101.0 x   103
    98.5 x    26        |          101.5 x    29
    98.0 x    88        |          102.0 x    32
    97.5 x    66        |          102.5 x    94
    97.0 x    27        |          103.0 x    84

  (Queued stops: 1)
[t=002] IN: STOP_LIMIT SELL qty=12 @ 99.0 (stop=99.0) [IOC]
     -> queued a